In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

try:
    df = pd.read_csv('training_set.csv')
    print("✅ Training dataset loaded successfully.")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print("❌ Error: 'training_set.csv' not found. Please place it in the correct directory.")
    exit()

✅ Training dataset loaded successfully.
Dataset shape: (1981520, 79)


In [ ]:

print("\nMissing values per column:")
print(df.isnull().sum())

for col in df.select_dtypes(include=np.number).columns:
    df[col].fillna(df[col].mean(), inplace=True)

X = df.drop(' Label', axis=1)
y = df[' Label']


Missing values per column:
 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64


/tmp/ipython-input-3970738545.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [ ]:
from sklearn.preprocessing import MinMaxScaler 

X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].median(), inplace=True)

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

print("✅ Features have been normalized using Min-Max scaling (range 0 to 1).")
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

/tmp/ipython-input-2320834879.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].median(), inplace=True)


✅ Features have been normalized using Min-Max scaling (range 0 to 1).


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

y = y.astype(int)

weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = dict(enumerate(weights))

print(f"\nCalculated class weights: {class_weights}")
print("This will penalize errors on the minority class more heavily.")

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y,
    test_size=0.2, 
    random_state=42,
    stratify=y 
)


model = tf.keras.Sequential([
  
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()


early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,          
    verbose=1,           
    restore_best_weights=True 
)

model_checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_loss',      
    save_best_only=True,    
    verbose=1               
)


print("\n--- Starting Model Training with Early Stopping and Checkpointing ---")

history = model.fit(
    X_train, y_train,
    epochs=5, 
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

print("--- Model Training Finished ---")


Calculated class weights: {0: np.float64(0.6226620947630923), 1: np.float64(2.5381194409148664)}
This will penalize errors on the minority class more heavily.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │         5,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,169 (28.00 KB)

...

 Non-trainable params: 0 (0.00 B)


--- Starting Model Training with Early Stopping and Checkpointing ---
Epoch 1/5
49520/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9302 - loss: 0.1543 - precision_2: 0.7634 - recall_2: 0.9534
Epoch 1: val_loss improved from inf to 0.07100, saving model to best_model.h5


49538/49538 ━━━━━━━━━━━━━━━━━━━━ 189s 4ms/step - accuracy: 0.9302 - loss: 0.1543 - precision_2: 0.7634 - recall_2: 0.9534 - val_accuracy: 0.9724 - val_loss: 0.0710 - val_precision_2: 0.8909 - val_recall_2: 0.9801
Epoch 2/5
49534/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9646 - loss: 0.0893 - precision_2: 0.8635 - recall_2: 0.9743
Epoch 2: val_loss did not improve from 0.07100
49538/49538 ━━━━━━━━━━━━━━━━━━━━ 176s 4ms/step - accuracy: 0.9646 - loss: 0.0893 - precision_2: 0.8635 - recall_2: 0.9743 - val_accuracy: 0.9683 - val_loss: 0.0776 - val_precision_2: 0.8687 - val_recall_2: 0.9884
Epoch 3/5
49522/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9683 - loss: 0.0808 - precision_2: 0.8744 - recall_2: 0.9789
Epoch 3: val_loss improved from 0.07100 to 0.06818, saving model to best_model.h5


49538/49538 ━━━━━━━━━━━━━━━━━━━━ 179s 4ms/step - accuracy: 0.9683 - loss: 0.0808 - precision_2: 0.8744 - recall_2: 0.9789 - val_accuracy: 0.9770 - val_loss: 0.0682 - val_precision_2: 0.9036 - val_recall_2: 0.9886
Epoch 4/5
49522/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9703 - loss: 0.0767 - precision_2: 0.8816 - recall_2: 0.9807
Epoch 4: val_loss did not improve from 0.06818
49538/49538 ━━━━━━━━━━━━━━━━━━━━ 177s 4ms/step - accuracy: 0.9703 - loss: 0.0767 - precision_2: 0.8816 - recall_2: 0.9807 - val_accuracy: 0.9683 - val_loss: 0.0751 - val_precision_2: 0.8694 - val_recall_2: 0.9875
Epoch 5/5
49530/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9710 - loss: 0.0738 - precision_2: 0.8839 - recall_2: 0.9822
Epoch 5: val_loss improved from 0.06818 to 0.06514, saving model to best_model.h5


49538/49538 ━━━━━━━━━━━━━━━━━━━━ 186s 4ms/step - accuracy: 0.9710 - loss: 0.0738 - precision_2: 0.8840 - recall_2: 0.9822 - val_accuracy: 0.9707 - val_loss: 0.0651 - val_precision_2: 0.8795 - val_recall_2: 0.9866
Restoring model weights from the end of the best epoch: 5.
--- Model Training Finished ---


In [ ]:
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int)

print("\n--- Model Evaluation on Validation Set ---")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

12385/12385 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step

--- Model Evaluation on Validation Set ---
Confusion Matrix:
[[307679  10555]
 [  1044  77026]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    318234
           1       0.88      0.99      0.93     78070

    accuracy                           0.97    396304
   macro avg       0.94      0.98      0.96    396304
weighted avg       0.97      0.97      0.97    396304



In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

try:
    test_df = pd.read_csv('test_set.csv')
    print("✅ Hold-out test set loaded successfully.")
except FileNotFoundError:
    print("❌ Error: 'testing_set.csv' not found. Please check the file name.")
    exit()

# --- 2. Separate Features and Labels ---
X_test = test_df.drop(' Label', axis=1)
y_test = test_df[' Label']


# --- 3. Apply IDENTICAL Preprocessing Steps ---
print("--- Applying preprocessing to the test set ---")

# a. Align columns using the columns from the ORIGINAL training DataFrame 'X'
# This is the fix for your AttributeError.
train_cols = X.columns
X_test = X_test.reindex(columns=train_cols, fill_value=0)

# b. Clean any infinite values and fill NaNs
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
# Use the median calculated from the TRAINING data's DataFrame 'X'
X_test.fillna(X.median(), inplace=True)

# c. Normalize the features using the SAME scaler fitted during training
X_test_scaled = scaler.transform(X_test)

print("✅ Test set preprocessed identically.")


# --- 4. Make Predictions and Evaluate ---
print("\n--- Generating Final Performance Report ---")

# Use the final, trained 'model' to predict on the prepared test data
y_test_pred_proba = model.predict(X_test_scaled)
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

# Print the final classification report
print("\nFINAL Classification Report on Unseen Test Data:")
print(classification_report(y_test, y_test_pred))

# Print the final confusion matrix
print("\nFINAL Confusion Matrix on Unseen Test Data:")
print(confusion_matrix(y_test, y_test_pred))

✅ Hold-out test set loaded successfully.
--- Applying preprocessing to the test set ---
✅ Test set preprocessed identically.

--- Generating Final Performance Report ---
26539/26539 ━━━━━━━━━━━━━━━━━━━━ 37s 1ms/step

FINAL Classification Report on Unseen Test Data:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    681929
           1       0.88      0.99      0.93    167294

    accuracy                           0.97    849223
   macro avg       0.94      0.98      0.96    849223
weighted avg       0.97      0.97      0.97    849223


FINAL Confusion Matrix on Unseen Test Data:
[[659897  22032]
 [  2287 165007]]


In [13]:
import tensorflow as tf

# List all physical devices visible to TensorFlow
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"✅ GPU is available. TensorFlow will automatically use it.")
    for gpu in gpus:
        print(f"   - {gpu}")
else:
    print("⚠️ No GPU found. TensorFlow will use the CPU.")

✅ GPU is available. TensorFlow will automatically use it.
   - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


### SECOND MODEL

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load your dataset
# Replace 'your_multiclass_dataset.csv' with the name of your file
try:
    df = pd.read_csv('secondary_classification_dataset.csv')
    print("✅ Dataset loaded successfully.")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print("❌ Error: 'your_multiclass_dataset.csv' not found. Please check the file name.")
    exit()

✅ Dataset loaded successfully.
Dataset shape: (557646, 79)


In [7]:
df[' Label'].value_counts()

,count
Label,
DoS Hulk,231073
PortScan,158930
DDoS,128027
DoS GoldenEye,10293
FTP-Patator,7938
SSH-Patator,5897
DoS slowloris,5796
DoS Slowhttptest,5499
Bot,1966


In [11]:
print("--- Original Class Distribution ---")
print(df[' Label'].value_counts())

# Define the mapping for the classes you want to group
# We'll map the specific web attacks to a general 'Web_Attack' category
web_attack_map = {
    'Web Attack � Brute Force': 'Web_Attack',
    'Web Attack � XSS': 'Web_Attack',
    'Web Attack � Sql Injection': 'Web_Attack'
}

# You can also create a map for very rare attacks
rare_attack_map = {
    'Infiltration': 'Rare_Attack',
    'Heartbleed': 'Rare_Attack'
}

# Apply the replacements to the 'Label1' column
df[' Label'] = df[' Label'].replace(web_attack_map)
df[' Label'] = df[' Label'].replace(rare_attack_map)

print("\n--- New Class Distribution After Grouping ---")
print(df[' Label'].value_counts())

# Now you can proceed with preprocessing this modified DataFrame

--- Original Class Distribution ---
 Label
DoS Hulk                      231073
PortScan                      158930
DDoS                          128027
DoS GoldenEye                  10293
FTP-Patator                     7938
SSH-Patator                     5897
DoS slowloris                   5796
DoS Slowhttptest                5499
Bot                             1966
Web Attack � Brute Force        1507
Web Attack � XSS                 652
Rare_Attack                       47
Web Attack � Sql Injection        21
Name: count, dtype: int64

--- New Class Distribution After Grouping ---
 Label
DoS Hulk            231073
PortScan            158930
DDoS                128027
DoS GoldenEye        10293
FTP-Patator           7938
SSH-Patator           5897
DoS slowloris         5796
DoS Slowhttptest      5499
Web_Attack            2180
Bot                   1966
Rare_Attack             47
Name: count, dtype: int64


In [12]:
import pandas as pd

# Assume 'df' is your DataFrame after the first round of grouping
# with the ' Label' column.

print("--- Distribution Before Final Grouping ---")
print(df[' Label'].value_counts())

# Define the new mapping dictionaries
dos_map = {
    'DoS Hulk': 'DoS_Attack',
    'DDoS': 'DoS_Attack',
    'DoS GoldenEye': 'DoS_Attack',
    'DoS slowloris': 'DoS_Attack',
    'DoS Slowhttptest': 'DoS_Attack'
}

brute_force_map = {
    'FTP-Patator': 'Brute_Force',
    'SSH-Patator': 'Brute_Force'
}

other_map = {
    'Bot': 'Other_Attack',
    'Rare_Attack': 'Other_Attack'
}

# Apply the replacements sequentially
df[' Label'] = df[' Label'].replace(dos_map)
df[' Label'] = df[' Label'].replace(brute_force_map)
df[' Label'] = df[' Label'].replace(other_map)

print("\n--- Final, Improved Class Distribution ---")
print(df[' Label'].value_counts())

--- Distribution Before Final Grouping ---
 Label
DoS Hulk            231073
PortScan            158930
DDoS                128027
DoS GoldenEye        10293
FTP-Patator           7938
SSH-Patator           5897
DoS slowloris         5796
DoS Slowhttptest      5499
Web_Attack            2180
Bot                   1966
Rare_Attack             47
Name: count, dtype: int64

--- Final, Improved Class Distribution ---
 Label
DoS_Attack      380688
PortScan        158930
Brute_Force      13835
Web_Attack        2180
Other_Attack      2013
Name: count, dtype: int64


In [13]:
# --- Data Cleaning ---
# Fill missing numerical values with the column median
for col in df.select_dtypes(include=np.number).columns:
    df[col].fillna(df[col].median(), inplace=True)
print("✅ Missing values handled.")

# --- Separate Features (X) and Labels (y) ---
X = df.drop(' Label', axis=1)
y = df[' Label']

# --- Label Encoding for Multiclass ---
# First, convert text labels to integer labels if they aren't already
y, class_names = pd.factorize(y)
num_classes = len(class_names)
print(f"Found {num_classes} unique classes.")

/tmp/ipython-input-1765854896.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


✅ Missing values handled.
Found 5 unique classes.


In [14]:
# --- Label Encoding for Multiclass ---
# First, convert text labels to integer labels if they aren't already
y, class_names = pd.factorize(y)
num_classes = len(class_names)
print(f"Found {num_classes} unique classes.")

# Second, one-hot encode the integer labels
# This converts a single column of labels [0, 1, 2] into three columns:
# [[1,0,0], [0,1,0], [0,0,1]]
y_one_hot = tf.keras.utils.to_categorical(y, num_classes=num_classes)
print("✅ Labels have been one-hot encoded.")

Found 5 unique classes.
✅ Labels have been one-hot encoded.


In [16]:
from sklearn.preprocessing import MinMaxScaler # Import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import numpy as np

# --- Data Cleaning: Handle infinities and NaNs in X ---
# Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill remaining NaN values with the median of each column
# (Using the median is robust to outliers which might be present due to large values)
for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].median(), inplace=True)
print("✅ Infinite and missing values in features handled.")


# --- Feature Scaling ---
# Apply Min-Max Scaling instead of Standard Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features have been normalized using Min-Max scaling.")

# --- Handle Class Imbalance ---
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights_dict = dict(enumerate(class_weights))
print(f"Calculated class weights: {class_weights_dict}")

# --- Create Training and Validation Sets ---
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_one_hot,
    test_size=0.2, # 20% for validation
    random_state=42,
    stratify=y # Stratify based on the original integer labels
)
print(f"Data split into {len(X_train)} training and {len(X_val)} validation samples.")

/tmp/ipython-input-2633925225.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].median(), inplace=True)


✅ Infinite and missing values in features handled.
✅ Features have been normalized using Min-Max scaling.
Calculated class weights: {0: np.float64(0.2929674694237801), 1: np.float64(0.7017504561756748), 2: np.float64(55.40447093889717), 3: np.float64(51.16018348623853), 4: np.float64(8.06138055655945)}
Data split into 446116 training and 111530 validation samples.


In [18]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# --- Build the Neural Network ---
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    # KEY CHANGE: Output layer for multiclass
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# --- Compile the Model ---
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    # ADD PRECISION AND RECALL TO THE METRICS LIST
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

model.summary()

# --- Define Callbacks for Efficient Training ---
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath='best_multiclass_model.h5', monitor='val_loss', save_best_only=True)

# --- Train the Model ---
print("\n--- Starting Model Training ---")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)
print("--- Model Training Finished ---")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        10,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,693 (73.02 KB)

 Trainable params: 18,693 (73.02 KB)

 Non-trainable params: 0 (0.00 B)

...

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 52s 4ms/step - accuracy: 0.9561 - loss: 0.4542 - precision: 0.9740 - recall: 0.9005 - val_accuracy: 0.9911 - val_loss: 0.0643 - val_precision: 0.9926 - val_recall: 0.9908
Epoch 2/20
13941/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9892 - loss: 0.1028 - precision: 0.9906 - recall: 0.9876

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 44s 3ms/step - accuracy: 0.9892 - loss: 0.1028 - precision: 0.9906 - recall: 0.9876 - val_accuracy: 0.9952 - val_loss: 0.0383 - val_precision: 0.9954 - val_recall: 0.9951
Epoch 3/20
13935/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9908 - loss: 0.0947 - precision: 0.9917 - recall: 0.9895

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - accuracy: 0.9908 - loss: 0.0946 - precision: 0.9917 - recall: 0.9895 - val_accuracy: 0.9962 - val_loss: 0.0234 - val_precision: 0.9962 - val_recall: 0.9958
Epoch 4/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - accuracy: 0.9911 - loss: 0.0977 - precision: 0.9919 - recall: 0.9901 - val_accuracy: 0.9953 - val_loss: 0.0363 - val_precision: 0.9955 - val_recall: 0.9953
Epoch 5/20
13939/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9891 - loss: 0.0967 - precision: 0.9906 - recall: 0.9871

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - accuracy: 0.9891 - loss: 0.0967 - precision: 0.9906 - recall: 0.9871 - val_accuracy: 0.9968 - val_loss: 0.0142 - val_precision: 0.9971 - val_recall: 0.9967
Epoch 6/20
13936/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9910 - loss: 0.0993 - precision: 0.9921 - recall: 0.9899

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 45s 3ms/step - accuracy: 0.9910 - loss: 0.0993 - precision: 0.9921 - recall: 0.9899 - val_accuracy: 0.9964 - val_loss: 0.0142 - val_precision: 0.9965 - val_recall: 0.9962
Epoch 7/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 45s 3ms/step - accuracy: 0.9896 - loss: 0.0958 - precision: 0.9912 - recall: 0.9879 - val_accuracy: 0.9954 - val_loss: 0.0317 - val_precision: 0.9957 - val_recall: 0.9954
Epoch 8/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 48s 3ms/step - accuracy: 0.9894 - loss: 0.1399 - precision: 0.9904 - recall: 0.9883 - val_accuracy: 0.9959 - val_loss: 0.0251 - val_precision: 0.9960 - val_recall: 0.9954
Epoch 9/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - accuracy: 0.9914 - loss: 0.0770 - precision: 0.9926 - recall: 0.9901 - val_accuracy: 0.9964 - val_loss: 0.0158 - val_precision: 0.9969 - val_recall: 0.9963
Epoch 10/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.9911 - loss: 0.0828 - precision: 0.9926 - recall: 0.9895 - val_accuracy: 0.9964

In [21]:
# --- Evaluate the Model ---
# The model outputs probabilities for each class
y_pred_proba = model.predict(X_val)

# To get the final predicted class, we find the index of the highest probability
y_pred = np.argmax(y_pred_proba, axis=1)

# The validation labels are one-hot encoded, so we convert them back for the report
y_true = np.argmax(y_val, axis=1)

# --- Print Evaluation Metrics ---
print("\n--- Model Evaluation on Validation Set ---")

# 1. Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# 2. Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

3486/3486 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step

--- Model Evaluation on Validation Set ---
Confusion Matrix:
[[75862     1     2   124   149]
 [   60 31722     0     4     0]
 [    1     0   401     0     1]
 [    6     0     0   391    39]
 [    6     2     0     2  2757]]

Classification Report:
              precision    recall  f1-score   support

  DoS_Attack       1.00      1.00      1.00     76138
    PortScan       1.00      1.00      1.00     31786
Other_Attack       1.00      1.00      1.00       403
  Web_Attack       0.75      0.90      0.82       436
 Brute_Force       0.94      1.00      0.97      2767

    accuracy                           1.00    111530
   macro avg       0.94      0.98      0.95    111530
weighted avg       1.00      1.00      1.00    111530



In [22]:
# Assume 'model' is your trained Keras model
model.save('my_multiclass_model.keras')

print("✅ Model saved successfully to 'my_multiclass_model.keras'")

✅ Model saved successfully to 'my_multiclass_model.keras'


In [23]:
!ls -l

total 190788
-rw-r--r-- 1 root root    257736 Oct 11 14:03 best_multiclass_model.h5
-rw-r--r-- 1 root root    253995 Oct 11 14:13 my_multiclass_model.keras
drwxr-xr-x 1 root root      4096 Oct  9 13:36 sample_data
-rw-r--r-- 1 root root 194841054 Oct 11 13:37 secondary_classification_dataset.csv
